# [SETUP] 


In [1]:
spark

In [2]:
%%sql
use prod.db

25/07/04 22:53:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

# [What] Use window functions to do looping with SQL

The reason for window functions is because historically, SQL has been set based i.e supporting operating on a set of rows at a time (e.g. GROUP BY,).

 

Lets see why we may need window functions as opposed to a `GROUP BY`.

![GROUP BY](../../images/groupby.png)

![WINDOW FUNCTION](../../images/window.png)

**NOTE** Notice how `GROUP BY` changes granularity, i.e. the input data had one row per order (aka order grain or order level) but the output had one row per date (aka date grain or date level).

When you want to operate on a set of rows without changing the grain `Window functions` are almost always a good fit.


# [WHY] Common scenarios when you want to use window functions

1. Calculate running metrics/sliding window over rows in the table (aggregate functions)
2. Ranking rows based on values in column(s) (ranking functions)
3. Access other row's values while operating on the current row (value functions)
4. Any combination of the above

## [HOW] to create a window function



### Window functions have four essential parts

1. **Partition**: Defines a set of rows based on specified column(s) value. If no partition is specified, the entire table is considered a partition.
2. **Order By**: This optional clause specifies how to order the rows within a partition.
3. **Function**: The function to be applied on the current row. The function results in an additional column in the output.
4. **Window frame**: Within a partition, a window frame allows you to specify the rows to be considered in the function computation.


![](../../images/framing.svg)

![Create window function](../../images/create_window.png)


In [3]:
%%sql
SELECT
  o_custkey,
  o_orderdate,
  o_totalprice,
  SUM(o_totalprice) -- FUNCTION 
  OVER (
    PARTITION BY
      o_custkey -- PARTITION
    ORDER BY
      o_orderdate -- ORDER BY; ASCENDING ORDER unless specified as DESC
  ) AS running_sum
FROM
  orders
WHERE
  o_custkey = 4
ORDER BY
  o_orderdate
LIMIT
  10;

o_custkey,o_orderdate,o_totalprice,running_sum
4,1992-06-09,261208.46,261208.46
4,1992-07-16,206615.30,467823.76
4,1992-10-11,222977.75,690801.51
4,1993-08-13,80130.69,770932.20
4,1993-11-13,66417.98,837350.18
4,1994-03-02,274992.65,1112342.83
4,1994-04-29,199636.40,1311979.23
4,1995-05-13,107127.51,1419106.74
4,1995-08-28,282207.37,1701314.11
4,1995-09-18,181789.20,1883103.31


The function `SUM` that we use in the above query is an aggregate function. Notice how the `running_sum` adds up (aka aggregates) the `o_totalprice` over all the rows. The rows themselves are ordered in ascending order by its orderdate.

**Reference**: While the standard aggregate functions are `MIN, MAX, AVG, SUM, & COUNT`, modern data systems offer a variety of powerful aggregations functions. Check your database documentation for available aggreagate functions. [e.g. list of agg functions available in duckdb](https://duckdb.org/docs/sql/functions/aggregates.html#general-aggregate-functions)

### [Exercise] 

Write a query to calculate the daily running average of totalprice of every customer. 

Time limit during live workshop: 5 min

**Hint**: Figure out the `PARTITION BY` column first, then the `ORDER BY` column and finally the `FUNCTION` to use to compute running average.


In [4]:
%%sql
SELECT
  o_custkey,
  o_orderdate,
  o_totalprice,
  AVG(o_totalprice) -- FUNCTION 
  OVER (
    PARTITION BY
      o_custkey -- PARTITION
    ORDER BY
      o_orderdate -- ORDER BY; ASCENDING ORDER unless specified as DESC
  ) AS running_sum
FROM
  orders
WHERE
  o_custkey = 4
ORDER BY
  o_orderdate
LIMIT
  10;


o_custkey,o_orderdate,o_totalprice,running_sum
4,1992-06-09,261208.46,261208.460000
4,1992-07-16,206615.30,233911.880000
4,1992-10-11,222977.75,230267.170000
4,1993-08-13,80130.69,192733.050000
4,1993-11-13,66417.98,167470.036000
4,1994-03-02,274992.65,185390.471667
4,1994-04-29,199636.40,187425.604286
4,1995-05-13,107127.51,177388.342500
4,1995-08-28,282207.37,189034.901111
4,1995-09-18,181789.20,188310.331000


## [HOW] to use ranking functions to get top/bottom n rows

If you are working on a problem to get the top/bottom n rows (as defined by some value) then use the **row** functions.

Let's look at an example of how to use a row function:

From the `orders` table **get the top 3 spending customers per day**. The orders table schema is shown below:

![Orders table](../../images/orders.png)


### [Example]

In [5]:
%%sql
SELECT
  *
FROM
  (
    SELECT
      o_orderdate,
      o_totalprice,
      o_custkey,
      RANK() -- RANKING FUNCTION 
      OVER (
        PARTITION BY
          o_orderdate -- PARTITION BY order date
        ORDER BY
          o_totalprice DESC -- ORDER rows withing partition by totalprice
      ) AS rnk
    FROM
      orders
  )
WHERE
  rnk <= 3
ORDER BY
  o_orderdate
LIMIT
  5;

o_orderdate,o_totalprice,o_custkey,rnk
1992-01-01,358534.43,10942,1
1992-01-01,307146.32,5426,2
1992-01-01,306169.25,12973,3
1992-01-02,319926.05,12161,1
1992-01-02,302943.34,10526,2


### [Exercise] From the `orders` table get the 3 lowest spending customers per day 

**Time limit during live workshop: 5 min**

**Hint**
  1. Figure out the `PARTITION BY` column first, then the `ORDER BY` column and finally the `FUNCTION` to use to compute running average.

The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [6]:
%%sql 
SELECT
  *
FROM
  (
    SELECT
      o_orderdate,
      o_totalprice,
      o_custkey,
      RANK() -- RANKING FUNCTION 
      OVER (
        PARTITION BY
          o_orderdate -- PARTITION BY order date
        ORDER BY
          o_totalprice -- ORDER rows withing partition by totalprice
      ) AS rnk
    FROM
      orders
  )
WHERE
  rnk <= 3
ORDER BY
  o_orderdate
LIMIT
  5;


o_orderdate,o_totalprice,o_custkey,rnk
1992-01-01,12781.38,9376,1
1992-01-01,25496.77,9103,2
1992-01-01,30944.67,14218,3
1992-01-02,16908.46,2746,1
1992-01-02,25445.96,4355,2



## [REFERENCE] standard RANKING functions: 

1. **`RANK`**: Ranks the rows starting from 1 to n within the window frame. Ranks the rows with the same value (defined by the "ORDER BY" clause) as the same and skips the ranking numbers that would have been present if the values were different.

2. **`DENSE_RANK`**: Ranks the rows starting from 1 to n within the window frame. Ranks the rows with the same value (defined by the "ORDER BY" clause) as the same and does not skip any ranking numbers.

3. **`ROW_NUMBER`**: Adds a row number that starts from 1 to n within the window frame and does not create any repeating values.



### [Example]

In [7]:
%%sql
-- Let's look at an example showing the difference between RANK, DENSE_RANK and ROW_NUMBER
SELECT 
    order_date,
    order_id,
    total_price,
    ROW_NUMBER() OVER (PARTITION BY order_date ORDER BY total_price) AS row_number,
    RANK() OVER (PARTITION BY order_date ORDER BY total_price) AS rank,
    DENSE_RANK() OVER (PARTITION BY order_date ORDER BY total_price) AS dense_rank
FROM (
    SELECT 
        '2024-07-08' AS order_date, 'order_1' AS order_id, 100 AS total_price UNION ALL
    SELECT 
        '2024-07-08', 'order_2', 200 UNION ALL
    SELECT 
        '2024-07-08', 'order_3', 150 UNION ALL
    SELECT 
        '2024-07-08', 'order_4', 90 UNION ALL
    SELECT 
        '2024-07-08', 'order_5', 100 UNION ALL
    SELECT 
        '2024-07-08', 'order_6', 90 UNION ALL
    SELECT 
        '2024-07-08', 'order_7', 100 UNION ALL
    SELECT 
        '2024-07-10', 'order_8', 100 UNION ALL
    SELECT 
        '2024-07-10', 'order_9', 100 UNION ALL
    SELECT 
        '2024-07-10', 'order_10', 100 UNION ALL
    SELECT 
        '2024-07-11', 'order_11', 100
) AS orders
ORDER BY order_date, row_number;


order_date,order_id,total_price,row_number,rank,dense_rank
2024-07-08,order_4,90,1,1,1
2024-07-08,order_6,90,2,1,1
2024-07-08,order_1,100,3,3,2
2024-07-08,order_5,100,4,3,2
2024-07-08,order_7,100,5,3,2
2024-07-08,order_3,150,6,6,3
2024-07-08,order_2,200,7,7,4
2024-07-10,order_8,100,1,1,1
2024-07-10,order_9,100,2,1,1
2024-07-10,order_10,100,3,1,1


Now that we have see how to **define a window function** and how to use **ranking and aggregation** functions, let's take it a step further by practicing **value functions**.

Remember that value functions are used to access other row's values while operating on the current row

Let's take a look at LEAD and LAG functions:

![LAG AND LEAD](../../images/leadlag.png)

### [Exercise] Write a SQL query using the `orders` table that calculates the following columns:
	1. o_orderdate: From orders table
	2. o_custkey: From orders table
	3. o_totalprice: From orders table
	4. totalprice_diff: The customers current day's o_totalprice - that same customers most recent previous purchase's o_totalprice

* **Time limit during live workshop: 5 min**

* **Hint**: 
    1. Start by figuring out what the `PARTITION BY` column should be, then what the `ORDER BY` column should be, and then finally the function to use. 
    2. Use the `LAG(column_name)` ranking function to identify the prior day's revenue.

* The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [8]:
%%sql
SELECT
  o_orderdate,
  o_custkey,
  o_totalprice,
  o_totalprice - lag (o_totalprice) OVER (
    PARTITION BY
      o_custkey
    ORDER BY
      o_orderdate
  ) AS totalprice_diff
FROM
  orders
ORDER BY
  o_custkey,
  o_orderdate
LIMIT
  50;

o_orderdate,o_custkey,o_totalprice,totalprice_diff
1992-04-19,1,83779.26,None
1992-04-26,1,283261.47,199482.21
1993-06-22,1,263247.54,-20013.93
1994-12-24,1,35523.05,-227724.49
1995-11-01,1,235155.22,199632.17
1996-05-08,1,81485.84,-153669.38
1996-12-09,1,45744.09,-35741.75
1996-12-26,1,11925.85,-33818.24
1997-03-04,1,268835.44,256909.59
1992-12-07,2,195693.26,None


## [REFERENCE] Here is a quick definition of the standard VALUE functions: 

1. **NTILE(n)**: Divides the rows in the window frame into n approximately equal groups, and assigns a number to each row indicating which group it belongs to.
2. **FIRST_VALUE()**: Returns the first value in the window frame.
3. **LAST_VALUE()**: Returns the last value in the window frame.
4. **LAG()**: Accesses data from a previous row within the window frame.
5. **LEAD()**: Accesses data from a subsequent row within the window frame.


# [WHY] define a window frame when you have a partition?

While our functions operate on the rows in the partition a window frame provides more granular ways to operate on a select set of rows within a partition.

When we need to operate one a set of rows within a parition (e.g. a sliding window) we can use the window frame to define these set of rows.

![Three order Sliding window average](../../images/slidingwindow.png)




Example

Consider a scenario where you have sales data, and you want to calculate a 3-day moving average of sales within each store:

```sql
SELECT
    store_id,
    sale_date,
    sales_amount,
    AVG(sales_amount) OVER (
        PARTITION BY store_id
        ORDER BY sale_date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS moving_avg_sales
FROM
    sales;
```

In this example:

1. **PARTITION BY** store_id ensures the calculation is done separately for each store.
2. **ORDER BY** sale_date defines the order of rows within each partition.
3. **ROWS BETWEEN 2 PRECEDING AND CURRENT ROW** specifies the window frame, considering the current row and the two preceding rows to calculate the moving average.
add: image
Without defining the window frame, the function might not be able to provide the specific moving average calculation you need.

## [HOW] to define a window frame with ROWS

1. **ROWS**: Used to select a set of rows relative to the current row based on position.
    1. Row definition format `ROWS BETWEEN start_point AND end_point`.
    2. The start_point and end_point can be any of the following three (in the proper order:
        1. **n PRECEDING**: n rows preceding the current row. UNBOUNDED PRECEDING indicates all rows before the current row.
        2. **n FOLLOWING**: n rows following the current row. UNBOUNDED FOLLOWING indicates all rows after the current row.


Let's see how relative row numbers can be used to define a window range.

Consider this window function 

```sql
AVG(total_price) OVER ( -- FUNCTION: RUNNING AVERAGE
    PARTITION BY o_custkey -- PARTITIONED BY customer
    ORDER BY order_month 
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING -- WINDOW FRAME DEFINED AS 1 ROW PRECEDING to 1 ROW FOLLOWING
    )
```

![Window frame with ROWS](../../images/wf.png)

### [Example]


1. Write a query on the orders table that has the following output:
    1. o_custkey
    2. order_month: In YYYY-MM format, use strftime(o_orderdate, '%Y-%m') AS order_month
    3. total_price: Sum of o_totalprice for that month
    4. three_mo_total_price_avg: The 3 month (previous, current & next) average of total_price for that customer



In [11]:
%%sql
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER ( -- FUNCTION: RUNNING AVERAGE
      PARTITION BY
        o_custkey -- PARTITIONED BY customer
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING -- WINDOW FRAME DEFINED AS 1 ROW PRECEDING to 1 ROW FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      date_format(o_orderdate, 'yyyy-MM') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
LIMIT
  5;

order_month,o_custkey,total_price,three_mo_total_price_avg
1992-04,1,367040.73,315144.14
1993-06,1,263247.54,221937.11
1994-12,1,35523.05,177975.27
1995-11,1,235155.22,117388.04
1996-05,1,81485.84,124770.33


## [HOW] to define a window frame with RANGE

1. **RANGE**: Used to select a set of rows relative to the current row based on the value of the columns specified in the `ORDER BY` clause.
    1. Range definition format `RANGE BETWEEN start_point AND end_point`.
    2. The start_point and end_point can be any of the following:
        1. **CURRENT ROW**: The current row.
        2. **n PRECEDING**: All rows with values within the specified range that are less than or equal to n units preceding the value of the current row.
        3. **n FOLLOWING**: All rows with values within the specified range that are greater than or equal to n units following the value of the current row.
        4. **UNBOUNDED PRECEDING**: All rows before the current row within the partition.
        5. **UNBOUNDED FOLLOWING**: All rows after the current row within the partition.
    3. `RANGE` is particularly useful when dealing with numeric or date/time ranges, allowing for calculations like running totals, moving averages, or cumulative distributions.

Let's see how `RANGE` works with `AVG(total price) OVER (PARTITION BY customer id ORDER BY date RANGE BETWEEN INTERVAL '1' DAY PRECEDING AND '1' DAY FOLLOWING)` using the below visualization:

![RANGE](../../images/range.png)

### [Exercise]

Now that we have seen how to create a window frame with ROWS, let' explore how to do this with RANGE.

1. Write a query on the orders table that has the following output:
    1. order_month, 
    2. o_custkey,
    3. total_price,
    4. three_mo_total_price_avg
    5. **consecutive_three_mo_total_price_avg**: The consecutive 3 month average of total_price for that customer. Note that this should only include months that are chronologically next to each other.
                                                                                               
**Time limit during live workshop: 10 min**
                                                                                               
**Hint**: Use `CAST(strftime(o_orderdate, '%Y-%m-01') AS DATE)` to cast order_month to date format.

**Hint**: Use the `INTERVAL` format shown above to construct the window function to compute `consecutive_three_mo_total_price_avg` column.
                                                                                           
* The orders table schema is shown below:

![Orders table](../../images/orders.png)

In [12]:
%%sql
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        CAST(order_month AS DATE) RANGE BETWEEN INTERVAL '1' MONTH PRECEDING
        AND INTERVAL '1' MONTH FOLLOWING
    ),
    2
  ) AS consecutive_three_mo_total_price_avg,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
    date_format(o_orderdate, 'yyyy-MM-01') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
ORDER BY
  o_custkey,
  order_month
LIMIT
  50;


order_month,o_custkey,total_price,consecutive_three_mo_total_price_avg,three_mo_total_price_avg
1992-04-01,1,367040.73,367040.73,315144.14
1993-06-01,1,263247.54,263247.54,221937.11
1994-12-01,1,35523.05,35523.05,177975.27
1995-11-01,1,235155.22,235155.22,117388.04
1996-05-01,1,81485.84,81485.84,124770.33
1996-12-01,1,57669.94,57669.94,135997.07
1997-03-01,1,268835.44,268835.44,163252.69
1992-12-01,2,195693.26,195693.26,146273.09
1993-10-01,2,96852.91,96852.91,109659.98
1994-05-01,2,36433.77,36433.77,114186.37


25/07/04 22:59:18 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-58197459-08b0-4714-b88b-f8e9ae7878b0. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-58197459-08b0-4714-b88b-f8e9ae7878b0
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimize

# Recap

1. Use windows when:
	* Calculating running metrics (similar to GROUP BY, but keeping all the rows)
	* Rank rows based on specific column(s)
	* Access other row values from the current row
2. Window has four key parts: Partition, Order By, Function, Window Frame
3. Define window frame with ROWS or RANGE
4. Window functions are expensive; be mindful of the performance (Covered in another lesson)

# Helpers

1. Solutions are available at [workshop_solutions](./workshop_solutions.ipynb). **Note** You need to stop the kernel in this notebook before starting the next one, since DuckDB can only have one worker on it at a time.
2. Note the `outline`(or `Table of Contents` in the left pane on Jupyter notebook) is a easy way to navigate this workbook.

# Questions
